In [1]:
##################################
### Import the usual libraries ###
##################################

import numpy as np

import astropy
from astropy.io import fits, ascii
from astropy.table import Table, vstack, Column
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u

import matplotlib
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16}) #make plots more readable

import MagellanicStream as ms
import astropy.units as u
from astropy.coordinates import SkyCoord
from uncertainties import ufloat

import warnings
warnings.filterwarnings('ignore')

import glob

In [2]:
##############################
### User defined functions ###
##############################

def dups(data):
    list_data = list(data)
    keys = []
    values = []
    not_uniq = list(set([x for x in list_data if list_data.count(x)>1]))
    for i in range(len(not_uniq)):
        keys.append(not_uniq[i])
        values.append(np.where(data==not_uniq[i]))
    idx = dict(zip(keys, values))
    return not_uniq, idx

def absmag(magnitude,distance,par):
    ''' !!! Parallax must have units of mas and distances must have units of kpc!!! '''
    absm = []
    if par == True: #if distance is given as parallax
        dist = np.reciprocal(distance)*1000
        absm.append(magnitude-5.0*np.log10(dist)+5.0)
        absm = np.squeeze(np.array(absm))
    else: #if distance is given as distance
        absm.append(magnitude-5.0*np.log10(distance*1000)+5.0)
        absm = np.squeeze(np.array(absm))
    return absm

import pandas as pd #needed for readbasti
def readbasti(file):
    with open(file) as f:
        lines = f.readlines()
        met_age = lines[4].split(' ')
        met = float(met_age[9]) #9th element after line[4] split
        age = float(met_age[-1]) #last element after line[4] split
        df = pd.read_csv(file,comment='#',names=['MINI','MFIN','LOGL','LOGT','JMAG','HMAG','KMAG'],sep='\s+')
        table_out = Table.from_pandas(df)
        metcol = Column(np.ones(len(table_out))*met, name='FEH')
        table_out.add_column(metcol)
        agecol = Column(np.log10(np.ones(len(table_out))*(age*10**6)), name='LOGAGE')
        table_out.add_column(agecol)
        mas = df['MFIN']*(1.99*10**33) #mass in g
        tem = 10**df['LOGT'] #T in K
        lum = (10**df['LOGL'])*(3.9*10**33) #L in erg/s
        lggcol = Column(np.log10((6.67259*10**-8)*(mas)*4*np.pi*(5.67051*10**-5)*(tem**4)/(lum)),name='LOGG')
        table_out.add_column(lggcol)
    return table_out

def mad(dat):
    return np.median(np.absolute(dat - np.median(dat)))

def sal(MH,aM):
    return MH + np.log(0.638*(10**(aM))+0.362)

#######################
### Age of Universe ###
#######################

'''
@article{riess2019large,
  title={Large Magellanic Cloud Cepheid Standards Provide a 1\% Foundation for the Determination of the Hubble Constant and Stronger Evidence for Physics Beyond LambdaCDM},
  author={Riess, Adam G and Casertano, Stefano and Yuan, Wenlong and Macri, Lucas M and Scolnic, Dan},
  journal={arXiv preprint arXiv:1903.07603},
  year={2019}
}
'''

#Value 74.03 \pm 1.42 (km/s)/Mpc

H0 = 74.03*(u.km/u.s)/u.Mpc
hertz = H0.to(u.km/u.s/u.pc).to(u.km/u.s/u.km)
tage = (1/hertz).to(u.yr)
ageU = tage.value

In [12]:
####################################
### Import Globular Cluster Data ###
####################################

#Globular Cluster Star Data
filepath1 = '/Users/joshpovick/desktop/research/magellanic_ages/globulars/allStar-r12-l33beta_clean_nodup_gc.fits'
gcs = Table(fits.getdata(filepath1,0))

#Globular Cluster Params
filepath2 = '/Users/joshpovick/desktop/research/magellanic_ages/globulars/gc_param.txt'
df = pd.read_csv(filepath2,comment='#',names=['cluster','rah','ram','ras','decd','decm','decs','concen','r_c',\
                                              'r_t','feh','vhelio','sigma_v'],sep='\s+')

In [13]:
print(df)

   cluster   rah  ram     ras  decd  decm   decs  concen   r_c  r_t   feh  \
0    47TUC   0.0   24   5.670 -72.0     4  52.60    2.07  0.36  0.0 -0.72   
1       M2  21.0   33  27.020  -0.0    49  23.70    1.59  0.32  0.0 -1.65   
2       M3  13.0   42  11.620  28.0    22  38.20    1.89  0.37  0.0 -1.50   
3       M5  15.0   18  33.220   2.0     4  51.70    1.73  0.44  0.0 -1.29   
4      M12  16.0   47  14.180  -1.0    56  54.70    1.34  0.79  0.0 -1.37   
5      M13  16.0   41  41.634  36.0    27  40.75    1.53  0.62  0.0 -1.53   
6      M15  21.0   29  58.330  12.0    10   1.20    2.99  0.14  0.0 -2.37   
7      M53  13.0   12  55.250  18.0    10   5.40    1.72  0.35  0.0 -2.10   
8      M92  17.0   17   7.490  43.0     8   9.40    1.68  0.26  0.0 -2.31   
9     M107  16.0   32  31.860 -13.0     3  13.60    1.53  0.56  0.0 -1.02   
10   N4147  12.0   10   6.300  18.0    32  33.50    1.83  0.09  0.0 -1.80   
11   N5053  13.0   16  27.090  17.0    42   0.90    0.74  2.08  0.0 -2.27   

In [6]:
#####################
### PARSEC Models ###
#####################

'''
x1: logT
x2: K
x3: feh
x4: logg
'''

def modelparsec(x1,x2,x3,x4):
    p = [4.52271170e+02,-2.45756963e+02,1.23004992e+00,-8.43079619e-01,1.50107300e+01,\
         5.55614699e-01,2.33494763e-01,-6.40147708e+00,-6.73945112e-01,3.53816153e+01,1.33852966e-01,\
         9.73455106e-01]
    out = p[0]+p[1]*x1+p[2]*x2+p[3]*x3+p[4]*x4+p[5]*x1*x2+p[6]*x1*x3+p[7]*x1*x4+p[8]*x2*x4+p[9]*x1**2+\
    p[10]*x2**2+p[11]*x4**2 
    return out

def modelbasti(x1,x2,x3,x4):
    p = [5.23433476e+03,-3.97205558e+03,8.20841073e+01,-9.21763525e+01,\
         2.06637009e+01,-4.35505072e+01,4.87362445e+01,-8.22172772e+00,\
         -8.29707077e-01,-6.43772879e+00,6.05155923e+00 ,1.00029946e+03,\
         1.62862241e-01,1.13536509e+00,-8.29688025e+01,-3.89188238e-03]
    out = p[0]+p[1]*x1+p[2]*x2+p[3]*x3+p[4]*x4+p[5]*np.multiply(x1,x2)+p[6]*np.multiply(x1,x3)+\
    p[7]*np.multiply(x1,x4)+p[8]*np.multiply(x2,x4)+p[9]*np.multiply(x1**2,x3)+p[10]*np.multiply(x1**2,x2)+\
    p[11]*(x1**2)+p[12]*(x2**2)+p[13]*(x4**2)+p[14]*(x1**3)+p[15]*(x4**3)
    return out

In [10]:
set(gcs['CLUSTER'])

{'47TUC',
 'M10',
 'M107',
 'M12',
 'M13',
 'M15',
 'M2',
 'M3',
 'M4',
 'M5',
 'M53',
 'M55',
 'M68',
 'M71',
 'M79',
 'M92',
 'N1851',
 'N2808',
 'N3201',
 'N362',
 'N4147',
 'N5053',
 'N5466',
 'N6229',
 'N6388',
 'N6397',
 'N6752',
 'OCEN'}

2918